In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
#funcao
def f(x,y,z):
    return 10 * (x - 1)**2 + 20 * (y - 2)**2 + 30 * (z - 3)**2

#população
def criaPop3d(tamanho):
    pop = np.random.uniform(low=0 , high=1 , size = (tamanho,3))
    #converter valores
    lb = [-3,-2,0]
    ub = [3,4,6]
    x = lb[0]+(ub[0]-lb[0])*pop[:,:1]
    y = lb[1]+(ub[1]-lb[1])*pop[:,1:2]
    z = lb[2]+(ub[2]-lb[2])*pop[:,2:]

    #nova população com valores convertidos
    popconv = np.append(x.reshape(-1, 1), y.reshape(-1, 1), axis=1)
    popconv = np.append(popconv, z.reshape(-1, 1), axis=1)
    return popconv


## População 100

In [3]:
#população inicial
pop=criaPop3d(100)

#fitness
pop = np.append(pop,(f(pop[0:,0:1],pop[0:,1:2],pop[0:,2:])).reshape(-1, 1), axis=1)

#ordenação
pop_ord = pop[pop[0:, 3].argsort()]

#50% melhores
melhor_pop_inicial = pop_ord[0:50,:]
#print(pop_ord.shape,melhor_pop_inicial.shape)

In [5]:
#contagem de população estagnada
cont = 0

for i in range(500000):
    #segunda geracao
    if i==0:
        nova_melhor_pop = melhor_pop_inicial.copy()

        #nova população
        nova_pop = criaPop3d(50)

        #fitness da nova população
        nova_pop = np.append(nova_pop,(f(nova_pop[0:,0:1],nova_pop[0:,1:2],nova_pop[0:,2:])).reshape(-1, 1), axis=1)

        #melhor pop anterior + nova população
        nova_melhor_pop = np.append(nova_melhor_pop,(nova_pop),axis= 0)

        #reordena
        melhor_pop = nova_melhor_pop[nova_melhor_pop[0:, 3].argsort()]
        
        #comparação entre populações
        comparacao = np.array_equal(melhor_pop, melhor_pop_inicial)
        #contagem de populações semelhantes (!ta errado!)
        if comparacao == True:
            cont = cont+1
    
    #gerações posteriores
    elif i>0:
        nova_melhor_pop = melhor_pop[0:50,:]

        nova_pop = criaPop3d(50)

        nova_pop = np.append(nova_pop,(f(nova_pop[0:,0:1],nova_pop[0:,1:2],nova_pop[0:,2:])).reshape(-1, 1), axis=1)

        nova_melhor_pop = np.append(nova_melhor_pop,(nova_pop),axis= 0)

        melhor_pop = nova_melhor_pop[nova_melhor_pop[0:, 3].argsort()]
        
        comparacao = np.array_equal(nova_melhor_pop, melhor_pop)
        if comparacao == True:
            cont = cont+1

            
print(cont,'\n',melhor_pop)

0 
 [[ 9.99803519e-01  1.99129847e+00  3.00253017e+00  1.70677253e-03]
 [ 1.00025495e+00  1.99064040e+00  2.99767075e+00  1.91545591e-03]
 [ 9.88616097e-01  2.00200173e+00  2.99465172e+00  2.23419306e-03]
 [ 1.01410159e+00  2.00653937e+00  3.00368724e+00  3.25168733e-03]
 [ 9.84749633e-01  2.01241929e+00  2.99906819e+00  5.43656021e-03]
 [ 9.84887323e-01  2.00190818e+00  3.01042335e+00  5.61613992e-03]
 [ 1.02298587e+00  2.00880481e+00  2.99734013e+00  7.04623930e-03]
 [ 9.80577621e-01  2.00312531e+00  3.01564600e+00  1.13115638e-02]
 [ 9.82032188e-01  2.01750700e+00  2.99058811e+00  1.20158341e-02]
 [ 9.94116672e-01  2.02364167e+00  3.00486674e+00  1.22352632e-02]
 [ 9.68784704e-01  2.00543172e+00  3.01088019e+00  1.38853769e-02]
 [ 9.68494593e-01  2.00055933e+00  2.98732288e+00  1.47534485e-02]
 [ 1.01862954e+00  1.97842169e+00  3.01115377e+00  1.65152640e-02]
 [ 9.78058486e-01  1.98060186e+00  2.98689888e+00  1.74892301e-02]
 [ 1.01482337e+00  2.02072379e+00  2.98498988e+00  1.75459